# バッチ正規化(batch normalization)レイヤ

In [11]:
import numpy as np

## バッチ正規化(batch normalization)レイヤの計算手順
ここでの説明は、バッチ正規化レイヤの入力層側に設置されている層の出力値のうちの1ノード分を対象にする。  

### [計算手順(学習時の順伝播計算)] 
#### (1) 計算の対象をxとする  
$~~~$入力 :  ${\bf x} = \{x_1,x_2, \dots , x_n\}$  
$~~~$n : データ数=バッチサイズ  
  
  
#### (2) 入力の平均値を求める    
$~~~$$\displaystyle \mu = \frac{1}{n}\sum_{i=1}^{n}x_i$
  
  
#### (3) 入力の分散を求める  
$~~~$$\displaystyle \sigma^2 = \frac{1}{n}\sum_{i=1}^{n}(x_i-\mu)^2$
  
  
#### (4) 入力を標準化する  
$~~~$各入力値について以下の処理を行う。numpyで計算する場合はベクトルとスカラーの演算が可能。  
$~~~$$\displaystyle \hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2+\epsilon}} $   
$~~~$$\epsilon$ : $1e-8$ (深層学習, Goodfellow, p.229)
      
      
#### (5) スケールし、平行移動させる  
$~~~$各入力値について以下の処理を行う。numpyで計算する場合はベクトルとスカラーの演算が可能。  
$~~~$$\displaystyle y_i = \gamma \hat{x}_i + \beta $  
$~~~$$y_i$が返り値になる。  
$~~~$$\gamma$と$\beta$は、標準化された$x$の分布を最適な分布に変換するための係数であり、学習の過程で最適化されていくパラメータ。1つのミニバッチ内で計算される平均$\mu$と分散$\sigma^2$とは値が異なる。


### [計算手順(予測時の順伝播計算)] 
基本的には、学習時の順伝播計算と同じだが、$\mu$と$\sigma^2$は、学習時に求めた移動平均値を使う
$~~~$  
$~~~$  

  
### [計算手順(学習時の逆伝播計算)] 
スライドの計算グラフを参照
$~~~$  
$~~~$  
  
  
[参考]
* 原著論文
    * https://arxiv.org/pdf/1502.03167.pdf
* ブログ
    * https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html

### [演習]
* 以下のバッチ正規化(batch normalization)レイヤクラスを完成させましょう.
* 入力xは、バッチ正規化レイヤの入力層側に設置されている層の出力値. n*d行列になっていることに注意.  

  入力 :  ${\bf x}=\quad
    \begin{pmatrix} 
    x_{11} & x_{12} & \dots & x_{1d}\\
    x_{21} & x_{22} & \dots & x_{2d}\\
   \vdots  & \vdots  & \ddots & \vdots \\
    x_{n1} & x_{n2} & \dots & x_{nd}\\
    \end{pmatrix}
    \quad$

    * ${\bf x}$ は、n*d行列
    * n : バッチサイズ  
    * d : 入力層側の層のノード数    

In [12]:
# ヒント
x = np.array([[1,2,3],[2,3,2],[3,1,4],[4,1,2]]) # N×D行列
print("x=",x)
mu = np.mean(x, axis=0) # 要素数D個のベクトル
print("mu=", mu)
var = np.mean((x-mu)**2, axis=0)  # 要素数D個のベクトル
print("var=", var)

x= [[1 2 3]
 [2 3 2]
 [3 1 4]
 [4 1 2]]
mu= [2.5  1.75 2.75]
var= [1.25   0.6875 0.6875]


In [13]:
class BatchNormalization:
    def __init__(self, gamma, beta, rho=0.9, moving_mean=None, moving_var=None):
        self.gamma = gamma # スケールさせるためのパラメータ, 学習によって更新させる.
        self.beta = beta # シフトさせるためのパラメータ, 学習によって更新させる
        self.rho = rho # 移動平均を算出する際に使用する係数

        # 予測時に使用する平均と分散
        self.moving_mean = moving_mean   # muの移動平均
        self.moving_var = moving_var     # varの移動平均
        
        # 計算中に算出される値を保持しておく変数群
        self.batch_size = None
        self.x_mu = None
        self.x_std = None        
        self.std = None
        self.dgamma = None
        self.dbeta = None

    def forward(self, x, train_flg=True):
        """
        順伝播計算
        x :  CNNの場合は4次元、全結合層の場合は2次元  
        """
        if x.ndim == 4:
            """
            画像形式の場合
            """
            N, C, H, W = x.shape
            x = x.transpose(0, 2, 3, 1) # NHWCに入れ替え
            x = x.reshape(N*H*W, C) # (N*H*W,C)の2次元配列に変換
            out = self.__forward(x, train_flg)
            out = out.reshape(N, H, W, C)# 4次元配列に変換
            out = out.transpose(0, 3, 1, 2) # 軸をNCHWに入れ替え
        elif x.ndim == 2:
            """
            画像形式以外の場合
            """
            out = self.__forward(x, train_flg)           
            
        return out
            
    def __forward(self, x, train_flg, epsilon=1e-8):
        """
        x : 入力. N×Dの行列. Nはバッチサイズ. Dは手前の層のノード数
        """
        if (self.moving_mean is None) or (self.moving_var is None):
            N, D = x.shape
            self.moving_mean = np.zeros(D)
            self.moving_var = np.zeros(D)
                        
        if train_flg:
            """
            学習時
            """
            # 入力xについて、Nの方向に平均値を算出. 
            mu = np.mean(x, axis=0) # 要素数D個のベクトル
            mu = np.broadcast_to(mu, (N, D)) # Nの方向にブロードキャスト
            print("mu.shape=", mu.shape)
            
            # 入力xから平均値を引く
            x_mu = x - mu   # N×D行列
            print("x_mu.shape=", x_mu.shape)
            
            # 入力xの分散を求める
            var = np.mean(x_mu**2, axis=0)  # 要素数D個のベクトル
            print("var.shape=", var.shape)
            
            # 入力xの標準偏差を求める(epsilonを足してから標準偏差を求める)
            std = np.sqrt(var + epsilon)  # 要素数D個のベクトル
            print("std.shape=", std.shape)
            
            # 標準偏差の逆数を求める
            std_inv = 1 / std
            std_inv = np.broadcast_to(std_inv, (N, D)) # Nの方向にブロードキャスト
            print("std_inv.shape=", std_inv.shape)
            
            # 標準化
            x_std = x_mu * std_inv  #N*D行列
            print("x_std.shape=", x_std.shape)
                  
            # 値を保持しておく
            self.batch_size = x.shape[0]
            self.x_mu = x_mu
            self.x_std = x_std
            self.std = std
            self.moving_mean = self.rho * self.moving_mean + (1-self.rho) * mu
            self.moving_var = self.rho * self.moving_var + (1-self.rho) * var            
        else:
            """
            予測時
            """
            x_mu = x - self.moving_mean # N*D行列
            x_std = x_mu / np.sqrt(self.moving_var + epsilon) # N*D行列
            
        # gammaでスケールし、betaでシフトさせる
        out = self.gamma * x_std + self.beta # N*D行列
        return out

    def backward(self, dout):
        """
        逆伝播計算
        dout : CNNの場合は4次元、全結合層の場合は2次元  
        """
        if dout.ndim == 4:
            """
            画像形式の場合
            """            
            N, C, H, W = dout.shape
            dout = dout.transpose(0, 2, 3, 1) # NHWCに入れ替え
            dout = dout.reshape(N*H*W, C) # (N*H*W,C)の2次元配列に変換
            dx = self.__backward(dout)
            dx = dx.reshape(N, H, W, C)# 4次元配列に変換
            dx = dx.transpose(0, 3, 1, 2) # 軸をNCHWに入れ替え
        elif dout.ndim == 2:
            """
            画像形式以外の場合
            """
            dx = self.__backward(dout)

        return dx

    def __backward(self, dout):
        """
        ここを完成させるには、計算グラフを理解する必要があり、実装にかなり時間がかかる.
        """
        N, D = self.x_mu.shape
        
        # betaの勾配
        dbeta = np.sum(dout, axis=0)
        
        # gammaの勾配(Nの方向に合計)
        dgamma = np.sum(self.x_std * dout, axis=0)
        
        # Xstdの勾配
        a1 = self.gamma * dout
        print("a1.shape=", a1.shape)
        
        # Xmuの勾配(1つ目)
        a2 = a1 / self.std
        print("a2.shape=", a2.shape)
        
        # 標準偏差の逆数の勾配
        a3 = a1 * self.x_mu
        print("a3.shape=", a3.shape)
        a3 = np.sum(a3, axis=0) # Nの方向に合計
        
        # 標準偏差の勾配
        a4 = -(a3) / (self.std * self.std)
        print("a4.shape=", a4.shape)
        
        # 分散の勾配
        a5 = 0.5 * a4 / self.std
        print("a5.shape=", a5.shape)
        
        # Xmuの2乗の勾配
        a6 = a5 / self.batch_size
        a6 = np.broadcast_to(a6, (N, D)) # Nの方向にブロードキャスト
        print("a6=",a6)
        print("a6.shape=", a6.shape)
        
        # Xmuの勾配(2つ目)
        a7 = 2.0  * self.x_mu * a6
        print("a7.shape=", a7.shape)
        
        # muの勾配
        a8 = -(a2+a7)
        print("a8.shape=", a8.shape)
        a8 = np.sum(a8, axis=0) # Nの方向に合計

        # Xの勾配
        a9 = a8 / self.batch_size
        a9 = np.broadcast_to(a9, (N, D)) # Nの方向にブロードキャスト
        dx = a2 + a7 + a9
        print("a9.shape=", a9.shape)
        
        self.dgamma = dgamma
        self.dbeta = dbeta
        
        return dx

In [14]:
# 入力が2次元の場合
hidden_size = 3
gamma = np.ones(hidden_size)
beta = np.zeros(hidden_size)
bn =BatchNormalization(gamma, beta)
        
x = np.array([[1,2,3],[2,3,2],[3,4,4],[4,1,2]]) # n*d行列
print("入力x=")
print(x)
print()

print("学習時の順伝播計算")
print(bn.forward(x, train_flg=True))
print()
print("予測時の順伝播計算")
print(bn.forward(x, train_flg=False))
print()

print("勾配")
dout = np.array([[0.1,0.2,0.3],[0.2,0.3,0.2],[0.3,0.4,0.4],[0.4,0.1,0.2]]) 
print(dout)
print()
print("学習時の逆伝播計算")
print(bn.backward(dout))
print()

入力x=
[[1 2 3]
 [2 3 2]
 [3 4 4]
 [4 1 2]]

学習時の順伝播計算
mu.shape= (4, 3)
x_mu.shape= (4, 3)
var.shape= (3,)
std.shape= (3,)
std_inv.shape= (4, 3)
x_std.shape= (4, 3)
[[-1.34164078 -0.44721359  0.30151134]
 [-0.44721359  0.44721359 -0.90453403]
 [ 0.44721359  1.34164078  1.50755671]
 [ 1.34164078 -1.34164078 -0.90453403]]

予測時の順伝播計算
[[ 2.12132026  4.94974727 10.39274147]
 [ 4.94974727  7.77817428  6.57889139]
 [ 7.77817428 10.60660129 14.20659155]
 [10.60660129  2.12132026  6.57889139]]

勾配
[[0.1 0.2 0.3]
 [0.2 0.3 0.2]
 [0.3 0.4 0.4]
 [0.4 0.1 0.2]]

学習時の逆伝播計算
a1.shape= (4, 3)
a2.shape= (4, 3)
a3.shape= (4, 3)
a4.shape= (3,)
a5.shape= (3,)
a6= [[-0.04472136 -0.04472136 -0.06030227]
 [-0.04472136 -0.04472136 -0.06030227]
 [-0.04472136 -0.04472136 -0.06030227]
 [-0.04472136 -0.04472136 -0.06030227]]
a6.shape= (4, 3)
a7.shape= (4, 3)
a8.shape= (4, 3)
a9.shape= (4, 3)
[[-1.07331255e-09 -3.57770841e-10  4.38561909e-10]
 [-3.57770841e-10  3.57770841e-10 -1.31568584e-09]
 [ 3.57770841e-10  1.073

In [15]:
# 入力が4次元の場合
N = 2
C = 6
H = 3
W = 3
gamma = np.ones(C)
beta = np.zeros(C)
bn =BatchNormalization(gamma, beta)
        
x = np.arange(N*C*H*W).reshape(N, C, H, W) # NCHW配列
print("入力x=")
print(x)
print()

print("学習時の順伝播計算")
print(bn.forward(x, train_flg=True))
print()
print("予測時の順伝播計算")
print(bn.forward(x, train_flg=False))
print()

print("勾配")
dout = np.arange(N*C*H*W).reshape(N, C, H, W) / 10 # NCHW配列.　10は値を調整するための適当な数
print(dout)
print()
print("学習時の逆伝播計算")
print(bn.backward(dout))

print()

入力x=
[[[[  0   1   2]
   [  3   4   5]
   [  6   7   8]]

  [[  9  10  11]
   [ 12  13  14]
   [ 15  16  17]]

  [[ 18  19  20]
   [ 21  22  23]
   [ 24  25  26]]

  [[ 27  28  29]
   [ 30  31  32]
   [ 33  34  35]]

  [[ 36  37  38]
   [ 39  40  41]
   [ 42  43  44]]

  [[ 45  46  47]
   [ 48  49  50]
   [ 51  52  53]]]


 [[[ 54  55  56]
   [ 57  58  59]
   [ 60  61  62]]

  [[ 63  64  65]
   [ 66  67  68]
   [ 69  70  71]]

  [[ 72  73  74]
   [ 75  76  77]
   [ 78  79  80]]

  [[ 81  82  83]
   [ 84  85  86]
   [ 87  88  89]]

  [[ 90  91  92]
   [ 93  94  95]
   [ 96  97  98]]

  [[ 99 100 101]
   [102 103 104]
   [105 106 107]]]]

学習時の順伝播計算
mu.shape= (18, 6)
x_mu.shape= (18, 6)
var.shape= (6,)
std.shape= (6,)
std_inv.shape= (18, 6)
x_std.shape= (18, 6)
[[[[-1.14293401 -1.10606517 -1.06919633]
   [-1.03232749 -0.99545865 -0.95858981]
   [-0.92172097 -0.88485213 -0.84798329]]

  [[-1.14293401 -1.10606517 -1.06919633]
   [-1.03232749 -0.99545865 -0.95858981]
   [-0.92172097 -0.88485